In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import random
from collections import OrderedDict
import math

In [2]:
#Recursive function to generate Hadamard matrices of order equal to powers of two

H = {}
H[1] = torch.tensor([[1.0]])
def Had(n):
    if(n not in H):
        Hnby2 = Had(n//2)
        Hn = torch.ones(n, n)
        for i in range(n//2):
            for j in range(n//2):
                Hn[i][j] = Hnby2[i][j]
                Hn[i+n//2][j] = Hnby2[i][j]
                Hn[i][j+n//2] = Hnby2[i][j]
                Hn[i+n//2][j+n//2] = -Hnby2[i][j]
        H[n] = Hn
        return Hn
    return H[n]

In [3]:
# Generating n boolean functions in k variables
def generate_data(k, n):
    two_pow_k = pow(2, k)
    # Input validation
    if(n<two_pow_k):
        print(f"Need 2^{k} inputs atleast for the model to converge. Give a bigger number for n")
        return []
    if(n>pow(2, two_pow_k)):
        print(f"n is greater than 2^2^{k}. Generating max possible functions - 2^2^{k}")
    if(two_pow_k<=16):
#         For generating data of sufficient LI inputs:
        while(True):
            perm = torch.randperm(pow(2, two_pow_k))[:n]
            data = [[0.0]*(two_pow_k-len(bin(num)[2:]))+[float(i) for i in bin(num)[2:]] for num in perm]
            # replacing f with (-1)^f
            data = [[1.0 if el==0.0 else -1.0 for el in bool_fun] for bool_fun in data]
            if(np.linalg.matrix_rank(data)<two_pow_k):
                print(f'Rank ({np.linalg.matrix_rank(data)}) of data is not large enough. Generating again.')
            else:
                print(f'Data generated. Rank = {np.linalg.matrix_rank(data)}')
                break           
    else:
        # A maximum of 2^2^4 samples are generated (with samples) for any k>4
        data = [[1.0 if random.random()>0.5 else -1.0 for i in range(two_pow_k)] for i in range(n)]
        print("Data generated.")
    return data            

In [4]:
# The network takes the processed k variable Boolean function (-1/1 for True/False) as the input and returns its Walsh Spectrum
# F -> WalshSpec
# WalshSpec = F * W_t

# forward(x) returns the Walsh spectrum of the Boolean function x, predict(x) return the non-linearity once the model is trained

class NetworkModel(nn.Module):

    def __init__(self, k):
        two_pow_k = pow(2, k)
        super(NetworkModel, self).__init__()
        self.lin1 = nn.Linear(two_pow_k, two_pow_k, bias=False)

    def forward(self, x):
        # Linear function without activation
        x = self.lin1(x)
        return x
    
    def predict(self, x):
        with torch.no_grad():
            self.eval()
            two_pow_k = len(x)
            nl = torch.round(0.5*(two_pow_k - max(self(x))))
        return nl
        

In [5]:
def train_network(model, data, numberOfInputs, learningRate, batchSize, epochs, device, optimizer):
    two_pow_k = len(data[0])
    n_ep, n_step = 0, 0
    for ep in range(0, epochs):
        for step in range(math.ceil(numberOfInputs/batchSize)):
            optimizer.zero_grad()
            input = torch.tensor(data[step:step+batchSize])
            output = model(input)
            loss = F.mse_loss(torch.matmul(input, Had(two_pow_k)), output)
            loss.backward()
            optimizer.step()
            weight_dict = OrderedDict(model.named_parameters())
            weightFunction = weight_dict['lin1.weight'].T
            H_error = F.mse_loss(Had(two_pow_k), weightFunction).item()
            if(H_error<0.00001):
                print(f"No. of epochs = {ep}, no. of steps = {step+ep*math.ceil(numberOfInputs/batchSize)}")
                n_ep, n_step = ep, step+ep*math.ceil(numberOfInputs/batchSize)
                return n_ep, n_step
    return -1, epochs*math.ceil(numberOfInputs/batchSize)

In [13]:
# Please change the number of variables (k) in the Boolean function here
k = 3
two_pow_k = pow(2, k)
n = pow(2, two_pow_k)

print(k, two_pow_k, n)

3 8 256


In [14]:
# Additional inputs may not be required to change.
learningRate = 0.1
batchSize = two_pow_k
numberOfInputs = 1*n//4      # change to vary the fraction of function space used for training (default at 1/4th)
max_epochs = 500

device = torch.device('cpu')
model = None
data = generate_data(k, numberOfInputs)
model =  NetworkModel(k)
optimizer = optim.SGD(model.parameters(), lr=learningRate, momentum=0.9)
n_ep, n_steps = train_network(model, data, numberOfInputs, learningRate, batchSize, max_epochs, device, optimizer)
weightMatrix = OrderedDict(model.named_parameters())['lin1.weight'].T
print("Final weight matrix: \n", weightMatrix)

Data generated. Rank = 8
No. of epochs = 31, no. of steps = 249
Final weight matrix: 
 tensor([[ 0.9901,  0.9947,  0.9951,  1.0020,  0.9922,  0.9959,  0.9934,  1.0031],
        [ 0.9914, -1.0046,  0.9957, -0.9982,  0.9932, -1.0036,  0.9942, -0.9973],
        [ 0.9951,  0.9974, -1.0024, -0.9990,  0.9962,  0.9980, -1.0033, -0.9985],
        [ 1.0032, -0.9983, -0.9984,  0.9993,  1.0026, -0.9987, -0.9978,  0.9990],
        [ 0.9987,  0.9993,  0.9994,  1.0003, -1.0010, -1.0005, -1.0009, -0.9996],
        [ 1.0003, -0.9998,  1.0002, -1.0001, -0.9997,  1.0001, -0.9998,  0.9999],
        [ 0.9989,  0.9994, -1.0006, -0.9998, -1.0009, -1.0005,  0.9992,  1.0003],
        [ 0.9983, -1.0009, -1.0008,  1.0003, -1.0013,  0.9993,  0.9989, -0.9995]],
       grad_fn=<PermuteBackward0>)


In [15]:
# Predict the non-linearity by passing the processed Boolean function of the form (-1)^f

bf = torch.tensor([1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, -1.0])
model.predict(bf)

tensor(2.)